In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
train=pd.read_csv('train.csv')

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [33]:
train.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [7]:
train['Gender']=train['Gender'].fillna(train['Gender'].mode()[0])

In [10]:
train['Married']=train['Married'].fillna(train['Married'].mode()[0])

In [17]:
train['Dependents']=train['Dependents'].fillna(train['Dependents'].mode()[0])

In [21]:
train['Self_Employed'] = train.groupby(['Education'])['Self_Employed'].transform(lambda x: x.fillna(x.mode()[0]))

In [24]:
train['Loan_Amount_Term']=train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0])

In [26]:
train['LoanAmount']=train['LoanAmount'].fillna(train['LoanAmount'].mean())

In [32]:
train['Credit_History']=train['Credit_History'].fillna(train['Credit_History'].mode()[0])

In [34]:
train.Credit_History.value_counts()

1.0    525
0.0     89
Name: Credit_History, dtype: int64

In [35]:
train.drop('Loan_ID',axis=1,inplace=True)

In [36]:
train.shape

(614, 12)

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
Gender               614 non-null object
Married              614 non-null object
Dependents           614 non-null object
Education            614 non-null object
Self_Employed        614 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           614 non-null float64
Loan_Amount_Term     614 non-null float64
Credit_History       614 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(7)
memory usage: 57.6+ KB


In [51]:
columns=['Gender','Married','Dependents','Education','Self_Employed','Property_Area']

In [52]:
len(columns)

6

In [53]:
def category_onehot_multcols(multcolumns):
    df_final=final_df
    i=0
    for fields in multcolumns:
        
        print(fields)
        df1=pd.get_dummies(final_df[fields],drop_first=True)
        
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
       
        
    df_final=pd.concat([final_df,df_final],axis=1)
        
    return df_final

In [54]:
main_df=train.copy()

In [55]:
test=pd.read_csv('formulatedtest.csv')

In [56]:
test.shape

(367, 11)

In [57]:
train.shape

(614, 12)

In [58]:
final_df=pd.concat([train,test],axis=0)

In [59]:
final_df.shape

(981, 12)

In [60]:
final_df=category_onehot_multcols(columns)

Gender
Married
Dependents
Education
Self_Employed
Property_Area


In [61]:
final_df.shape

(981, 15)

In [62]:
final_df =final_df.loc[:,~final_df.columns.duplicated()]

In [66]:
final_df.head()

,ApplicantIncome,CoapplicantIncome,Credit_History,LoanAmount,Loan_Amount_Term,Loan_Status,Male,Yes,1,2,3+,Not Graduate,Semiurban,Urban
0,5849,0.0,1.0,146.412162,360.0,1.0,1,0,0,0,0,0,0,1
1,4583,1508.0,1.0,128.000000,360.0,0.0,1,1,1,0,0,0,0,0
2,3000,0.0,1.0,66.000000,360.0,1.0,1,1,0,0,0,0,0,1
3,2583,2358.0,1.0,120.000000,360.0,1.0,1,1,0,0,0,1,0,1
4,6000,0.0,1.0,141.000000,360.0,1.0,1,0,0,0,0,0,0,1


In [64]:
final_df["Loan_Status"]=final_df.Loan_Status.map({'Y':1,'N':0})

In [67]:
df_train=final_df.iloc[:614,:]
df_test=final_df.iloc[614:,:]

In [68]:
df_train.shape

(614, 14)

In [69]:
df_test.shape

(367, 14)

In [70]:
df_test.drop('Loan_Status',axis=1,inplace=True)

In [71]:
df_test.shape

(367, 13)

In [72]:
X_train=df_train.drop('Loan_Status',axis=1)
y_train=df_train['Loan_Status']

# Model training

In [107]:
from sklearn.linear_model import LogisticRegression

In [108]:
lm=LogisticRegression()

In [109]:
lm.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [110]:
# # Create regularization penalty space
# penalty = ['l1', 'l2']

# # Create regularization hyperparameter space
# C = np.logspace(0, 4, 10)

# # Create hyperparameter options
# hyperparameters = dict(C=C, penalty=penalty)

# # Create grid search using 5-fold cross validation
# clf = GridSearchCV(lm, hyperparameters, cv=5, verbose=0)

# # Fit grid search
# best_model = clf.fit(X_train, y_train)

# # View best hyperparameters
# print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
# print('Best C:', best_model.best_estimator_.get_params()['C'])

In [111]:
# lm=LogisticRegression(penalty='l1',C='1.0')

In [112]:
# lm.fit(X_train,y_train)

In [113]:
y_pred=lm.predict(df_test)

In [114]:
pred=pd.DataFrame(y_pred)

In [115]:
sub_df=pd.read_csv('sample_submission.csv')

In [116]:
dataset=pd.concat([sub_df['Loan_ID'],pred],axis=1)

In [117]:
dataset.columns=['Loan_ID','Loan_Status']

In [118]:
dataset["Loan_Status"]=dataset.Loan_Status.map({1.0:'Y',0.0:'N'})

In [119]:
dataset.to_csv('submission01.csv',index=False)

# Random Forest

In [120]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
#making the instance
model=RandomForestClassifier()
#hyper parameters set
params = {'criterion':['gini','entropy'],
          'n_estimators':[10,15,20,25,30],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[3,4,5,6,7], 
          'random_state':[123],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#learning
model1.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)


Best Hyper Parameters:
 {'criterion': 'gini', 'min_samples_leaf': 3, 'min_samples_split': 7, 'n_estimators': 25, 'n_jobs': -1, 'random_state': 123}


In [121]:
Rf=RandomForestClassifier(criterion='gini', min_samples_leaf=3, min_samples_split= 7, n_estimators= 25, n_jobs= -1, random_state= 123)

In [122]:
Rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=7,
                       min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

In [123]:
y_pred1=Rf.predict(df_test)

In [124]:
pred1=pd.DataFrame(y_pred1)

In [125]:
sub_df=pd.read_csv('sample_submission.csv')
dataset=pd.concat([sub_df['Loan_ID'],pred1],axis=1)
dataset.columns=['Loan_ID','Loan_Status']
dataset["Loan_Status"]=dataset.Loan_Status.map({1.0:'Y',0.0:'N'})
dataset.to_csv('submission02.csv',index=False)